In [22]:
from jupyter_dash import JupyterDash

In [23]:
import dash
from dash.dependencies import Input, Output
import dash_bio as dashbio
from dash import html, dcc

In [24]:
from waitress import create_server
import threading
import logging

class FlaskAppWorker(threading.Thread):
  _ready = threading.Event()
  daemon = True

  def __init__(self, flask) -> None:
    super().__init__()
    self.flask = flask
    self.is_shutdown = False

  def run(self):
    try:
      # idempotent if logging has already been set up
      logging.basicConfig()
      logging.getLogger("waitress").setLevel(logging.ERROR)

      desired_host = "127.0.0.1"
      desired_port = 5000
      self.server = create_server(
        self.flask,
        host=desired_host,
        port=desired_port,
        clear_untrusted_proxy_headers=True,
      )
      print("Serving on http://%s:%s" % (self.server.effective_host, self.server.effective_port))
      self._ready.set()
      self.server.run()

    except Exception:
      if not self.is_shutdown:
        raise

  def shutdown(self) -> None:
    self.is_shutdown = True
    sockets = list(self.server._map.values())  # type: ignore
    for socket in sockets:
      socket.handle_close()
    self.server.task_dispatcher.shutdown()


worker = None

In [84]:
app = JupyterDash("clustergram")

In [85]:
#df = pd.read_csv('https://git.io/clustergram_brain_cancer.csv').set_index('ID_REF')
#columns = list(df.columns.values)
columns = [v.split('::', 2)[-1] for v in df.columns.values]
rows = list(df.index)

app.layout = html.Div([
    "Rows to display",
    dcc.Dropdown(
        id='my-default-clustergram-input',
        options=[
            #{'label': row.split('::', 2)[-1], 'value': row} for row in rows
            {'label': row.split('::')[-1], 'value': row} for row in rows
        ],
        value=rows,
        multi=True
    ),

    html.Div(id='my-default-clustergram')
])

@app.callback(
    Output('my-default-clustergram', 'children'),
    Input('my-default-clustergram-input', 'value')
)
def update_clustergram(rows):
    if len(rows) < 2:
        return "Please select at least two rows to display."

    return dcc.Graph(figure=dashbio.Clustergram(
        data=df.loc[rows].values,
        row_labels=[row.split('::')[-1] for row in rows],
        column_labels=columns,
        #color_threshold={'row': 250,'col': 700},
        #hidden_labels='row',
        height=100*len(rows),
        width=25*len(columns),
        display_ratio=[0.1, 1.0],
        optimal_leaf_order = True,
        color_map = [[0.0, '#1010a0'], [1.0, '#ff4040']]
    ))


In [86]:
if worker is not None:
    worker.shutdown()
    worker.join()
    worker = None

sys:1: ResourceWarning:

Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fc1f7ef1040>



In [87]:
worker = FlaskAppWorker(app.server)
worker.start()

Serving on http://127.0.0.1:5000


In [ ]:
app._display_in_jupyter('http://localhost:5000', port=8050, mode='inline', width='100%', height=650)